In [1]:
# Import all dependencies
import os
import cv2
import time
import argparse
import numpy as np
import tensorflow as tf
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from distutils.version import StrictVersion

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')
%matplotlib inline

In [2]:
#Import helper utilities
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import download_data_from_gdrive


/home/ubuntu/obj_detection/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


Enter the id in the shareable link here:
1we6ZFNxppabVvJQCqMwCDJ9WSqE5f9OA
Enter the file name with extension here:
walmart_trim.mp4
Hello
object_detection/test_video/walmart/input/walmart_trim.mp4
Download 100%.


In [ ]:
CWD_PATH = os.getcwd()

# Path to frozen detection graph. This is the actual model that is used for the object detection.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
PATH_TO_CKPT = os.path.join('../../', MODEL_NAME, 'frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(CWD_PATH, 'object_detection', 'data', 'mscoco_label_map.pbtxt')

In [ ]:
NUM_CLASSES = 1

# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,
                                                            use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
def detect_objects(image_np):
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=7)
    return image_np

In [ ]:

# First test on images
PATH_TO_TEST_IMAGES_DIR = 'object_detection/my_test_images/walmart/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.png'.format(i)) for i in range(1, 4) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    print(im_width)
    print(im_height)
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
from PIL import Image
for image_path in TEST_IMAGE_PATHS:
    im = cv2.imread(image_path)
    im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#     image = Image.open(image_path)
#     image_np = load_image_into_numpy_array(image)
#     plt.imshow(image_np)
    plt.imshow(im_rgb)
    print(im_rgb.size, im_rgb.shape)

In [ ]:
#Load a frozen TF model 
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:
#             image = Image.open(image_path)
#             image_np = load_image_into_numpy_array(image)
            im = cv2.imread(image_path)
            image_np = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            image_process = detect_objects(image_np, sess, detection_graph)
            print(image_process.shape)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_process)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import *
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # you should return the final output (image with lines are drawn on lanes)
    with tf.Session(graph=detection_graph) as sess:
        image_process = detect_objects(image, sess, detection_graph)
        return image_process

In [ ]:
white_output = 'test_video3_out.mp4'
clip1 = VideoFileClip("./object_detection/my_test_video/walmart/input/walmart_not_trimmed3.mp4")
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        white_clip = clip1.fl_image(detect_objects) #NOTE: this function expects color images!!s
        %time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="800" height="600" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
HTML("""
<video width="800" height="600" controls>
  <source src="./object_detection/my_test_video/walmart/input/walmart_not_trimmed3.mp4">
</video>
""")